# Introduction to Traces

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

Weave is a toolkit for developing AI-powered applications.

Use Weave traces to capture the inputs, outputs, and internal structure of your Python function automatically to observe and debug LLM applications.

When you decorate a function with `@weave.op`, Weave records a rich trace of the function while it runs, including any nested operations or external API calls. Use the trace to to debug, understand, and visualize interactions between your code and LLM models, without leaving your notebook.

To get started, complete the prerequisites. Then, define a function decorated with `@weave.op` decorator and run it on an example input to track LLM calls. Weave captures and visualizes the trace automatically.

In [ ]:
# Ensure your dependencies are installed with:
!pip install --quiet jedi openai weave

In [ ]:
import os
import getpass

#@title Set up your credentials
inference_provider = "W&B Inference" #@param ["W&B Inference", "OpenAI"]

# Set up your W&B project and credentials
os.environ["WANDB_ENTITY_PROJECT"] = input("Set up your W&B project (team name/project name): ")
os.environ["WANDB_API_KEY"] = getpass.getpass("Set up your W&B API key (Find it at https://wandb.ai/authorize): ")

# Set up your OpenAI API key
if inference_provider == "OpenAI":
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key (Find it at https://platform.openai.com/api-keys): ")

In [ ]:
from openai import OpenAI
import weave
import time

weave.init(os.environ["WANDB_ENTITY_PROJECT"])

@weave.op
def preprocess_message(message: str) -> str:
    """预处理用户输入"""
    time.sleep(0.1)  # 模拟处理时间
    processed = message.strip().lower()
    return processed

@weave.op
def setup_client(inference_provider: str) -> tuple:
    """设置OpenAI客户端"""
    time.sleep(0.05)  # 模拟初始化时间

    if inference_provider == "W&B Inference":
        client = OpenAI(
            base_url="https://api.inference.wandb.ai/v1",
            api_key=os.environ["WANDB_API_KEY"],
            project=os.environ["WANDB_ENTITY_PROJECT"],
        )
        model_name = "OpenPipe/Qwen3-14B-Instruct"
    else:
        client = OpenAI()
        model_name = "gpt-4.1-nano"

    return client, model_name

@weave.op
def call_llm(client, model_name: str, messages: list) -> str:
    """调用LLM API"""
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
    )
    return response.choices[0].message.content

@weave.op
def postprocess_response(response: str) -> str:
    """后处理响应"""
    time.sleep(0.02)  # 模拟处理时间
    # 添加一些格式化
    formatted = f"🤖 {response}"
    return formatted

@weave.op
def create_completion(message: str, inference_provider: str = "OpenAI") -> str:
    """主流程 - 创建完整的AI响应"""

    # 步骤1: 预处理
    processed_message = preprocess_message(message)

    # 步骤2: 设置客户端
    client, model_name = setup_client(inference_provider)

    # 步骤3: 准备消息
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": processed_message},
    ]

    # 步骤4: 调用LLM
    raw_response = call_llm(client, model_name, messages)

    # 步骤5: 后处理
    final_response = postprocess_response(raw_response)

    return final_response

# 测试调用
message = "Tell me a joke."
result = create_completion(message)
print(result)